In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tqdm

# 글자 전처리를 위한 import
import string
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator 

/opt/anaconda3/envs/dipLearning/lib/python3.12/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/anaconda3/envs/dipLearning/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/anaconda3/envs/dipLearning/lib/python3.12/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORT

In [2]:
data = pd.read_csv("netflix_reviews.csv")

In [3]:
# 전처리 함수
import re

def preprocess_text(text):
    if isinstance(text, float):
        return ""
    text = text.lower()  # 대문자를 소문자로
    text = re.sub(r'[^\w\s]', '', text)  # 구두점 제거
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = text.strip()  # 띄어쓰기 제외하고 빈 칸 제거
    return text

In [4]:
data["content"] = data["content"].apply(preprocess_text)
data["content"]

0                                                 cant open
1                                              the best app
2         most of the famous korean drama are not dubbed...
3         its superb but can you please add comments sec...
4         the only reason i didnt give it four stars is ...
                                ...                        
116926    i really like it there are so many movies and ...
116927       i love netflix i always enjoy my time using it
116928                 sound quality is very slow of movies
116929    rate is very expensive bcos we see netflix sun...
116930    this app is awesome for english movies series ...
Name: content, Length: 116931, dtype: object

In [5]:
X = data["content"].tolist()
y = data["score"].tolist()

train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(X, y, test_size=0.2, random_state=42)

# 토큰 생성
tokenizer = get_tokenizer('basic_english')

def yield_tokens(sentences):
    for text in sentences:
        yield tokenizer(text)

In [6]:
# 한 문장당 바꿔주는 iterator임
vocab = build_vocab_from_iterator(yield_tokens(train_reviews), specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])

def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

def label_pipeline(label):
    return int(label)-1

In [7]:
class ReviewDataset(Dataset):

    def __init__(self, reviews, ratings, text_pipeline, label_pipeline):
        self.reviews = reviews
        self.ratings = ratings
        self.text_pipeline = text_pipeline
        self.label_pipeline = label_pipeline

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.text_pipeline(self.reviews[idx])
        rating = self.label_pipeline(self.ratings[idx])
        
        return torch.tensor(review), torch.tensor(rating)

In [8]:
train_dataset = ReviewDataset(train_reviews, train_ratings, text_pipeline, label_pipeline)
test_dataset = ReviewDataset(test_reviews, test_ratings, text_pipeline, label_pipeline)

In [9]:
from torch.nn.utils.rnn import pad_sequence

MAX_LEN = 100

def collate_batch(batch):
    reviews, raitings = [], []
    
    for review, raiting in batch:
        # 최대 문장길이를 넘어가는 단어는 제거합니다.
        truncated_review = review[:MAX_LEN]
        reviews.append(torch.tensor(truncated_review, dtype=torch.long))
        raitings.append(torch.tensor(raiting, dtype=torch.long))

    # padding을 주어 짧은 문장에 대한 길이를 맞춥니다.
    padded_reviews = pad_sequence(reviews, batch_first=True, padding_value=0)

    if padded_reviews.size(1) > MAX_LEN:
        padded_reviews = padded_reviews[:, :MAX_LEN]
    
    return padded_reviews, torch.tensor(raitings, dtype=torch.long)

In [10]:
# 데이터 로더 정의
BATCH_SIZE = 64

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [11]:
len(set(y))

5

In [12]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.Wa = nn.Linear(hidden_dim, hidden_dim)
        self.Ua = nn.Linear(hidden_dim, hidden_dim)
        self.Va = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_output):
        batch_size, seq_len, _ = lstm_output.size()
        hidden = lstm_output[:, -1, :]

        score = self.Va(torch.tanh(self.Wa(lstm_output) + self.Ua(hidden).unsqueeze(1)))
        attention_weights = torch.softmax(score, dim=1)

        context_vector = torch.sum(attention_weights * lstm_output, dim=1)
        
        return context_vector

In [15]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.attention = Attention(hidden_dim) 
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)  
        context_vector = self.attention(output)
        return self.fc(context_vector)

# 하이퍼파라미터 수정
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64  
HIDDEN_DIM = 128  
OUTPUT_DIM = len(set(y))
NUM_EPOCHS = 100

# 모델 초기화
model = LSTMModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)
LABEL_SMOOTHING = 0.05

# 손실 함수와 옵티마이저 정의
# 라벨 스무딩을 이용하여 모델의 일반화 성능을 향상시킴 (평점이 리뷰와 안맞는 케이스 완화)
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [41]:
# 학습 루프 개선
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for reviews, ratings in tqdm.tqdm(train_dataloader):
        outputs = model(reviews)
        optimizer.zero_grad()
        loss = criterion(outputs, ratings)  
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{100}], Loss: {running_loss / len(train_dataloader):.4f}')
        

print('Finished Training')

  0%|                                                  | 0/1462 [00:00<?, ?it/s]/var/folders/jd/z3tj001966qd1b1w4_jn3_zr0000gn/T/ipykernel_4031/4066590696.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reviews.append(torch.tensor(truncated_review, dtype=torch.long))
/var/folders/jd/z3tj001966qd1b1w4_jn3_zr0000gn/T/ipykernel_4031/4066590696.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  raitings.append(torch.tensor(raiting, dtype=torch.long))
100%|███████████████████████████████████████| 1462/1462 [18:03<00:00,  1.35it/s]


Epoch [1/100], Loss: 0.8553


100%|███████████████████████████████████████| 1462/1462 [18:05<00:00,  1.35it/s]


Epoch [2/100], Loss: 0.8399


100%|█████████████████████████████████████| 1462/1462 [1:13:21<00:00,  3.01s/it]


...

Epoch [98/100], Loss: 0.6704


100%|███████████████████████████████████████| 1462/1462 [13:02<00:00,  1.87it/s]


Epoch [99/100], Loss: 0.6663


100%|███████████████████████████████████████| 1462/1462 [13:09<00:00,  1.85it/s]

Epoch [100/100], Loss: 0.6655
Finished Training


In [42]:
# 예측 함수(예시)
def predict_review(model, review):
    model.eval()
    with torch.no_grad():
        tensor_review = torch.tensor(text_pipeline(review), dtype=torch.long).unsqueeze(0)
        output = model(tensor_review)
        prediction = output.argmax(1).item()
        return prediction + 1

In [44]:
# 새로운 리뷰에 대한 예측
new_review = "I hate this app."
predicted_score = predict_review(model, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 1


In [45]:
# 새로운 리뷰에 대한 예측
new_review = "I love the diversity of lanugauge of this app."
predicted_score = predict_review(model, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 5


In [46]:
new_review = "So Great"
predicted_score = predict_review(model, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 5


In [53]:
new_review = "While Netflix has a lot of content, the app itself is frustrating to use. It frequently lags or freezes, especially when switching profiles or resuming a video, which makes watching anything feel like a hassle. The recommendation algorithm also seems off, constantly showing me shows I have no interest in despite my viewing history. For the price, I expected a more stable experience, but the app’s glitches and poor suggestions make it hard to fully enjoy. Hoping they fix these issues soon, but for now, it’s barely usable."
predicted_score = predict_review(model, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 3


In [54]:
new_review = "This app is great but has some bugs."
predicted_score = predict_review(model, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 4


In [57]:
new_review = "It was plain. Not impressed at all."
predicted_score = predict_review(model, new_review)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 1
